#Qubit ($d=2$) Dicke states

In [1]:
try:
    import cirq
except ImportError:
    print("installing cirq...")
    !pip install --quiet cirq
    import cirq

    print("installed cirq.")

installing cirq...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 579.1/579.1 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.2/58.2 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 596.4/596.4 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.0/221.0 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.7/151.7 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━

In [2]:
import numpy as np

In [3]:
simulator = cirq.Simulator()

In [ ]:
# from cirq.contrib.svg import SVGCircuit

In [4]:
#l=1
def ia_gate(qr,m):
  """Gives generator"""
  l=1
  theta=-2*np.arccos(np.sqrt(l/m))
  cry = cirq.ControlledGate(cirq.ry(theta),num_controls=1)
  yield cirq.CNOT(qr[0],qr[1])
  yield cry(qr[1],qr[0])
  yield cirq.CNOT(qr[0],qr[1])

#l>1
def ib_gate(qr,m,l):
  """Gives generator"""
  theta=-2*np.arccos(np.sqrt(l/m))
  ccry = cirq.ControlledGate(cirq.ry(theta),num_controls=2)
  yield cirq.CNOT(qr[0],qr[l])
  yield ccry(qr[l],qr[l-1],qr[0])
  yield cirq.CNOT(qr[0],qr[l])

def i_gate(qr,m,l):
  if l==1:
    yield ia_gate(qr,m)
  else:
    yield ib_gate(qr,m,l)


## NOT simplified

In [5]:
def W(qr,n,l,m):
  for j in range(1,m):
      yield i_gate(qr,m,j)

#W \otimes identity:
def WW(qr,n,l,m):
  yield W(qr[n-m:n],n,l,m)

def UU(qr,n,l):
  for m in reversed(range(2,n+1)):
    yield WW(qr,n,l,m)

def Dicke(qr,n,l):
  """Gives generator"""
  # initial state (identity permutation)
  for j in range(l):
    yield cirq.X(qr[j])
  # apply Dicke operator
  yield UU(qr,n,l)

In [6]:
n=2
l=1
qr=cirq.LineQubit.range(n)
test=cirq.Circuit(Dicke(qr,n,l))
print(test)


0: ───X───@───Ry(-0.5π)───@───
          │   │           │
1: ───────X───@───────────X───


In [7]:
result = simulator.simulate(test)
print(cirq.dirac_notation(result.final_state_vector))

0.71|01⟩ + 0.71|10⟩


In [8]:
n=3
l=1
qr=cirq.LineQubit.range(n)
test=cirq.Circuit(Dicke(qr,n,l))
print(test)

0: ───X───@───Ry(-0.608π)───@───@───Ry(-0.392π)───@───────────────────────
          │   │             │   │   │             │
1: ───────X───@─────────────X───┼───@─────────────┼───@───Ry(-0.5π)───@───
                                │   │             │   │   │           │
2: ─────────────────────────────X───@─────────────X───X───@───────────X───


In [9]:
result = simulator.simulate(test)
print(cirq.dirac_notation(result.final_state_vector))

0.58|001⟩ + 0.58|010⟩ + 0.58|100⟩


In [10]:
n=3
l=2
qr=cirq.LineQubit.range(n)
test=cirq.Circuit(Dicke(qr,n,l))
print(test)


0: ───X───@───Ry(-0.608π)───@───@───Ry(-0.392π)───@───────────────────────
          │   │             │   │   │             │
1: ───X───X───@─────────────X───┼───@─────────────┼───@───Ry(-0.5π)───@───
                                │   │             │   │   │           │
2: ─────────────────────────────X───@─────────────X───X───@───────────X───


In [11]:
result = simulator.simulate(test)
print(cirq.dirac_notation(result.final_state_vector))

0.58|011⟩ + 0.58|101⟩ + 0.58|110⟩


In [12]:
n=4
l=1
qr=cirq.LineQubit.range(n)
test=cirq.Circuit(Dicke(qr,n,l))
print(test)

                                                                      ┌──┐
0: ───X───@───Ry(-0.667π)───@───@───Ry(-0.5π)───@───@───Ry(-0.333π)────@─────────────────────────────────────────────────────────────────
          │   │             │   │   │           │   │   │              │
1: ───────X───@─────────────X───┼───@───────────┼───┼───┼──────────────┼@────Ry(-0.608π)───@───@───Ry(-0.392π)───@───────────────────────
                                │   │           │   │   │              ││    │             │   │   │             │
2: ─────────────────────────────X───@───────────X───┼───@──────────────┼X────@─────────────X───┼───@─────────────┼───@───Ry(-0.5π)───@───
                                                    │   │              │                       │   │             │   │   │           │
3: ─────────────────────────────────────────────────X───@──────────────X───────────────────────X───@─────────────X───X───@───────────X───
                                                  

In [13]:
result = simulator.simulate(test)
print(cirq.dirac_notation(result.final_state_vector))

0.5|0001⟩ + 0.5|0010⟩ + 0.5|0100⟩ + 0.5|1000⟩


In [14]:
n=4
l=2
qr=cirq.LineQubit.range(n)
test=cirq.Circuit(Dicke(qr,n,l))
print(test)

                                                                      ┌──┐
0: ───X───@───Ry(-0.667π)───@───@───Ry(-0.5π)───@───@───Ry(-0.333π)────@─────────────────────────────────────────────────────────────────
          │   │             │   │   │           │   │   │              │
1: ───X───X───@─────────────X───┼───@───────────┼───┼───┼──────────────┼@────Ry(-0.608π)───@───@───Ry(-0.392π)───@───────────────────────
                                │   │           │   │   │              ││    │             │   │   │             │
2: ─────────────────────────────X───@───────────X───┼───@──────────────┼X────@─────────────X───┼───@─────────────┼───@───Ry(-0.5π)───@───
                                                    │   │              │                       │   │             │   │   │           │
3: ─────────────────────────────────────────────────X───@──────────────X───────────────────────X───@─────────────X───X───@───────────X───
                                                  

In [15]:
result = simulator.simulate(test)
print(cirq.dirac_notation(result.final_state_vector))

0.41|0011⟩ + 0.41|0101⟩ + 0.41|0110⟩ + 0.41|1001⟩ + 0.41|1010⟩ + 0.41|1100⟩


In [16]:
n=4
l=3
qr=cirq.LineQubit.range(n)
test=cirq.Circuit(Dicke(qr,n,l))
print(test)

                                                                      ┌──┐
0: ───X───@───Ry(-0.667π)───@───@───Ry(-0.5π)───@───@───Ry(-0.333π)────@─────────────────────────────────────────────────────────────────
          │   │             │   │   │           │   │   │              │
1: ───X───X───@─────────────X───┼───@───────────┼───┼───┼──────────────┼@────Ry(-0.608π)───@───@───Ry(-0.392π)───@───────────────────────
                                │   │           │   │   │              ││    │             │   │   │             │
2: ───X─────────────────────────X───@───────────X───┼───@──────────────┼X────@─────────────X───┼───@─────────────┼───@───Ry(-0.5π)───@───
                                                    │   │              │                       │   │             │   │   │           │
3: ─────────────────────────────────────────────────X───@──────────────X───────────────────────X───@─────────────X───X───@───────────X───
                                                  

In [17]:
result = simulator.simulate(test)
print(cirq.dirac_notation(result.final_state_vector))

0.5|0111⟩ + 0.5|1011⟩ + 0.5|1101⟩ + 0.5|1110⟩


In [18]:
n=5
l=1
qr=cirq.LineQubit.range(n)
test=cirq.Circuit(Dicke(qr,n,l))
print(test)

                                                                        ┌──┐   ┌────────────┐   ┌────────────┐   ┌──┐                                     ┌──┐
0: ───X───@───Ry(-0.705π)───@───@───Ry(-0.564π)───@───@───Ry(-0.436π)────@──────@────────────────Ry(-0.295π)──────@──────────────────────────────────────────────────────────────────────────────────────────────────────────
          │   │             │   │   │             │   │   │              │      │                │                │
1: ───────X───@─────────────X───┼───@─────────────┼───┼───┼──────────────┼@─────┼Ry(-0.667π)─────┼──────────@─────┼@────Ry(-0.5π)───@───@───Ry(-0.333π)────@─────────────────────────────────────────────────────────────────
                                │   │             │   │   │              ││     ││               │          │     ││    │           │   │   │              │
2: ─────────────────────────────X───@─────────────X───┼───@──────────────┼X─────┼@───────────────┼──────────X─────┼┼────@───

In [19]:
result = simulator.simulate(test)
print(cirq.dirac_notation(result.final_state_vector))

0.45|00001⟩ + 0.45|00010⟩ + 0.45|00100⟩ + 0.45|01000⟩ + 0.45|10000⟩


In [20]:
n=5
l=2
qr=cirq.LineQubit.range(n)
test=cirq.Circuit(Dicke(qr,n,l))
print(test)

                                                                        ┌──┐   ┌────────────┐   ┌────────────┐   ┌──┐                                     ┌──┐
0: ───X───@───Ry(-0.705π)───@───@───Ry(-0.564π)───@───@───Ry(-0.436π)────@──────@────────────────Ry(-0.295π)──────@──────────────────────────────────────────────────────────────────────────────────────────────────────────
          │   │             │   │   │             │   │   │              │      │                │                │
1: ───X───X───@─────────────X───┼───@─────────────┼───┼───┼──────────────┼@─────┼Ry(-0.667π)─────┼──────────@─────┼@────Ry(-0.5π)───@───@───Ry(-0.333π)────@─────────────────────────────────────────────────────────────────
                                │   │             │   │   │              ││     ││               │          │     ││    │           │   │   │              │
2: ─────────────────────────────X───@─────────────X───┼───@──────────────┼X─────┼@───────────────┼──────────X─────┼┼────@───

In [21]:
result = simulator.simulate(test)
print(cirq.dirac_notation(result.final_state_vector))

0.32|00011⟩ + 0.32|00101⟩ + 0.32|00110⟩ + 0.32|01001⟩ + 0.32|01010⟩ + 0.32|01100⟩ + 0.32|10001⟩ + 0.32|10010⟩ + 0.32|10100⟩ + 0.32|11000⟩


In [22]:
n=5
l=3
qr=cirq.LineQubit.range(n)
test=cirq.Circuit(Dicke(qr,n,l))
print(test)

                                                                        ┌──┐   ┌────────────┐   ┌────────────┐   ┌──┐                                     ┌──┐
0: ───X───@───Ry(-0.705π)───@───@───Ry(-0.564π)───@───@───Ry(-0.436π)────@──────@────────────────Ry(-0.295π)──────@──────────────────────────────────────────────────────────────────────────────────────────────────────────
          │   │             │   │   │             │   │   │              │      │                │                │
1: ───X───X───@─────────────X───┼───@─────────────┼───┼───┼──────────────┼@─────┼Ry(-0.667π)─────┼──────────@─────┼@────Ry(-0.5π)───@───@───Ry(-0.333π)────@─────────────────────────────────────────────────────────────────
                                │   │             │   │   │              ││     ││               │          │     ││    │           │   │   │              │
2: ───X─────────────────────────X───@─────────────X───┼───@──────────────┼X─────┼@───────────────┼──────────X─────┼┼────@───

In [23]:
result = simulator.simulate(test)
print(cirq.dirac_notation(result.final_state_vector))

0.32|00111⟩ + 0.32|01011⟩ + 0.32|01101⟩ + 0.32|01110⟩ + 0.32|10011⟩ + 0.32|10101⟩ + 0.32|10110⟩ + 0.32|11001⟩ + 0.32|11010⟩ + 0.32|11100⟩


In [24]:
n=5
l=4
qr=cirq.LineQubit.range(n)
test=cirq.Circuit(Dicke(qr,n,l))
print(test)

                                                                        ┌──┐   ┌────────────┐   ┌────────────┐   ┌──┐                                     ┌──┐
0: ───X───@───Ry(-0.705π)───@───@───Ry(-0.564π)───@───@───Ry(-0.436π)────@──────@────────────────Ry(-0.295π)──────@──────────────────────────────────────────────────────────────────────────────────────────────────────────
          │   │             │   │   │             │   │   │              │      │                │                │
1: ───X───X───@─────────────X───┼───@─────────────┼───┼───┼──────────────┼@─────┼Ry(-0.667π)─────┼──────────@─────┼@────Ry(-0.5π)───@───@───Ry(-0.333π)────@─────────────────────────────────────────────────────────────────
                                │   │             │   │   │              ││     ││               │          │     ││    │           │   │   │              │
2: ───X─────────────────────────X───@─────────────X───┼───@──────────────┼X─────┼@───────────────┼──────────X─────┼┼────@───

In [25]:
result = simulator.simulate(test)
print(cirq.dirac_notation(result.final_state_vector))

0.45|01111⟩ + 0.45|10111⟩ + 0.45|11011⟩ + 0.45|11101⟩ + 0.45|11110⟩


In [26]:
n=6
l=3
qr=cirq.LineQubit.range(n)
test=cirq.Circuit(Dicke(qr,n,l))
print(test)

                                                                      ┌──┐   ┌────────────┐   ┌────────────┐   ┌──┐   ┌────────────┐   ┌────────────┐   ┌──┐                 ┌──┐   ┌────────────┐   ┌────────────┐   ┌──┐                                     ┌──┐
0: ───X───@───Ry(-0.732π)───@───@───Ry(-0.608π)───@───@───Ry(-0.5π)────@──────@────────────────Ry(-0.392π)──────@──────@────────────────Ry(-0.268π)──────@────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
          │   │             │   │   │             │   │   │            │      │                │                │      │                │                │
1: ───X───X───@─────────────X───┼───@─────────────┼───┼───┼────────────┼@─────┼Ry(-0.705π)─────┼──────────@─────┼@─────┼Ry(-0.564π)─────┼──────────@─────┼@────Ry(-0.436π)────@──────@────────────────Ry(-0.295π)──────@──────────────────────────────────────────────

In [27]:
result = simulator.simulate(test)
print(cirq.dirac_notation(result.final_state_vector))

0.22|000111⟩ + 0.22|001011⟩ + 0.22|001101⟩ + 0.22|001110⟩ + 0.22|010011⟩ + 0.22|010101⟩ + 0.22|010110⟩ + 0.22|011001⟩ + 0.22|011010⟩ + 0.22|011100⟩ + 0.22|100011⟩ + 0.22|100101⟩ + 0.22|100110⟩ + 0.22|101001⟩ + 0.22|101010⟩ + 0.22|101100⟩ + 0.22|110001⟩ + 0.22|110010⟩ + 0.22|110100⟩ + 0.22|111000⟩


## Simplified

In [28]:
def Wsimple(qr,n,l,m):
  for j in range(max(l+m-n,1),min(l,m-1)+1):
      yield i_gate(qr,m,j)

#W \otimes identity:
def WWsimple(qr,n,l,m):
  yield Wsimple(qr[n-m:n],n,l,m)

def UUsimple(qr,n,l):
  for m in reversed(range(2,n+1)):
    yield WWsimple(qr,n,l,m)

def DickeSimple(qr,n,l):
  """Gives generator"""
  # initial state (identity permutation)
  for j in range(l):
    yield cirq.X(qr[j])
  # apply Dicke operator
  yield UUsimple(qr,n,l)

In [29]:
n=2
l=1
qr=cirq.LineQubit.range(n)
test=cirq.Circuit(DickeSimple(qr,n,l))
print(test)


0: ───X───@───Ry(-0.5π)───@───
          │   │           │
1: ───────X───@───────────X───


In [30]:
result = simulator.simulate(test)
print(cirq.dirac_notation(result.final_state_vector))

0.71|01⟩ + 0.71|10⟩


In [31]:
n=3
l=1
qr=cirq.LineQubit.range(n)
test=cirq.Circuit(DickeSimple(qr,n,l))
print(test)


0: ───X───@───Ry(-0.608π)───@───────────────────────
          │   │             │
1: ───────X───@─────────────X───@───Ry(-0.5π)───@───
                                │   │           │
2: ─────────────────────────────X───@───────────X───


In [32]:
result = simulator.simulate(test)
print(cirq.dirac_notation(result.final_state_vector))

0.58|001⟩ + 0.58|010⟩ + 0.58|100⟩


In [33]:
n=3
l=2
qr=cirq.LineQubit.range(n)
test=cirq.Circuit(DickeSimple(qr,n,l))
print(test)


0: ───X───@───Ry(-0.392π)───@───────────────────────
          │   │             │
1: ───X───┼───@─────────────┼───@───Ry(-0.5π)───@───
          │   │             │   │   │           │
2: ───────X───@─────────────X───X───@───────────X───


In [34]:
result = simulator.simulate(test)
print(cirq.dirac_notation(result.final_state_vector))

0.58|011⟩ + 0.58|101⟩ + 0.58|110⟩


In [35]:
n=4
l=1
qr=cirq.LineQubit.range(n)
test=cirq.Circuit(DickeSimple(qr,n,l))
print(test)

0: ───X───@───Ry(-0.667π)───@─────────────────────────────────────────────
          │   │             │
1: ───────X───@─────────────X───@───Ry(-0.608π)───@───────────────────────
                                │   │             │
2: ─────────────────────────────X───@─────────────X───@───Ry(-0.5π)───@───
                                                      │   │           │
3: ───────────────────────────────────────────────────X───@───────────X───


In [36]:
result = simulator.simulate(test)
print(cirq.dirac_notation(result.final_state_vector))

0.5|0001⟩ + 0.5|0010⟩ + 0.5|0100⟩ + 0.5|1000⟩


In [37]:
n=4
l=2
qr=cirq.LineQubit.range(n)
test=cirq.Circuit(DickeSimple(qr,n,l))
print(test)

0: ───X───@───Ry(-0.5π)───@───────────────────────────────────────────────────────────────────
          │   │           │
1: ───X───┼───@───────────┼───@───Ry(-0.608π)───@───@───Ry(-0.392π)───@───────────────────────
          │   │           │   │   │             │   │   │             │
2: ───────X───@───────────X───X───@─────────────X───┼───@─────────────┼───@───Ry(-0.5π)───@───
                                                    │   │             │   │   │           │
3: ─────────────────────────────────────────────────X───@─────────────X───X───@───────────X───


In [38]:
result = simulator.simulate(test)
print(cirq.dirac_notation(result.final_state_vector))

0.41|0011⟩ + 0.41|0101⟩ + 0.41|0110⟩ + 0.41|1001⟩ + 0.41|1010⟩ + 0.41|1100⟩


In [39]:
n=4
l=3
qr=cirq.LineQubit.range(n)
test=cirq.Circuit(DickeSimple(qr,n,l))
print(test)

0: ───X───@───Ry(-0.333π)───@─────────────────────────────────────────────
          │   │             │
1: ───X───┼───┼─────────────┼───@───Ry(-0.392π)───@───────────────────────
          │   │             │   │   │             │
2: ───X───┼───@─────────────┼───┼───@─────────────┼───@───Ry(-0.5π)───@───
          │   │             │   │   │             │   │   │           │
3: ───────X───@─────────────X───X───@─────────────X───X───@───────────X───


In [40]:
result = simulator.simulate(test)
print(cirq.dirac_notation(result.final_state_vector))

0.5|0111⟩ + 0.5|1011⟩ + 0.5|1101⟩ + 0.5|1110⟩


In [41]:
n=5
l=1
qr=cirq.LineQubit.range(n)
test=cirq.Circuit(DickeSimple(qr,n,l))
print(test)

0: ───X───@───Ry(-0.705π)───@───────────────────────────────────────────────────────────────────
          │   │             │
1: ───────X───@─────────────X───@───Ry(-0.667π)───@─────────────────────────────────────────────
                                │   │             │
2: ─────────────────────────────X───@─────────────X───@───Ry(-0.608π)───@───────────────────────
                                                      │   │             │
3: ───────────────────────────────────────────────────X───@─────────────X───@───Ry(-0.5π)───@───
                                                                            │   │           │
4: ─────────────────────────────────────────────────────────────────────────X───@───────────X───


In [42]:
result = simulator.simulate(test)
print(cirq.dirac_notation(result.final_state_vector))

0.45|00001⟩ + 0.45|00010⟩ + 0.45|00100⟩ + 0.45|01000⟩ + 0.45|10000⟩


In [43]:
n=5
l=2
qr=cirq.LineQubit.range(n)
test=cirq.Circuit(DickeSimple(qr,n,l))
print(test)

0: ───X───@───Ry(-0.564π)───@─────────────────────────────────────────────────────────────────────────────────────────────────────────────
          │   │             │
1: ───X───┼───@─────────────┼───@───Ry(-0.667π)───@───@───Ry(-0.5π)───@───────────────────────────────────────────────────────────────────
          │   │             │   │   │             │   │   │           │
2: ───────X───@─────────────X───X───@─────────────X───┼───@───────────┼───@───Ry(-0.608π)───@───@───Ry(-0.392π)───@───────────────────────
                                                      │   │           │   │   │             │   │   │             │
3: ───────────────────────────────────────────────────X───@───────────X───X───@─────────────X───┼───@─────────────┼───@───Ry(-0.5π)───@───
                                                                                                │   │             │   │   │           │
4: ───────────────────────────────────────────────────────────────────────────────────────

In [44]:
result = simulator.simulate(test)
print(cirq.dirac_notation(result.final_state_vector))

0.32|00011⟩ + 0.32|00101⟩ + 0.32|00110⟩ + 0.32|01001⟩ + 0.32|01010⟩ + 0.32|01100⟩ + 0.32|10001⟩ + 0.32|10010⟩ + 0.32|10100⟩ + 0.32|11000⟩


In [45]:
n=5
l=3
qr=cirq.LineQubit.range(n)
test=cirq.Circuit(DickeSimple(qr,n,l))
print(test)

                                                                      ┌──┐
0: ───X───@───Ry(-0.436π)───@────────────────────────────────────────────────────────────────────────────────────────────────────────────
          │   │             │
1: ───X───┼───┼─────────────┼───@───Ry(-0.5π)───@───@───Ry(-0.333π)────@─────────────────────────────────────────────────────────────────
          │   │             │   │   │           │   │   │              │
2: ───X───┼───@─────────────┼───┼───@───────────┼───┼───┼──────────────┼@────Ry(-0.608π)───@───@───Ry(-0.392π)───@───────────────────────
          │   │             │   │   │           │   │   │              ││    │             │   │   │             │
3: ───────X───@─────────────X───X───@───────────X───┼───@──────────────┼X────@─────────────X───┼───@─────────────┼───@───Ry(-0.5π)───@───
                                                    │   │              │                       │   │             │   │   │           │
4: ─────────────────

In [46]:
result = simulator.simulate(test)
print(cirq.dirac_notation(result.final_state_vector))

0.32|00111⟩ + 0.32|01011⟩ + 0.32|01101⟩ + 0.32|01110⟩ + 0.32|10011⟩ + 0.32|10101⟩ + 0.32|10110⟩ + 0.32|11001⟩ + 0.32|11010⟩ + 0.32|11100⟩


In [47]:
n=5
l=4
qr=cirq.LineQubit.range(n)
test=cirq.Circuit(DickeSimple(qr,n,l))
print(test)

0: ───X───@───Ry(-0.295π)───@───────────────────────────────────────────────────────────────────
          │   │             │
1: ───X───┼───┼─────────────┼───@───Ry(-0.333π)───@─────────────────────────────────────────────
          │   │             │   │   │             │
2: ───X───┼───┼─────────────┼───┼───┼─────────────┼───@───Ry(-0.392π)───@───────────────────────
          │   │             │   │   │             │   │   │             │
3: ───X───┼───@─────────────┼───┼───@─────────────┼───┼───@─────────────┼───@───Ry(-0.5π)───@───
          │   │             │   │   │             │   │   │             │   │   │           │
4: ───────X───@─────────────X───X───@─────────────X───X───@─────────────X───X───@───────────X───


In [48]:
result = simulator.simulate(test)
print(cirq.dirac_notation(result.final_state_vector))

0.45|01111⟩ + 0.45|10111⟩ + 0.45|11011⟩ + 0.45|11101⟩ + 0.45|11110⟩


In [49]:
n=6
l=3
qr=cirq.LineQubit.range(n)
test=cirq.Circuit(DickeSimple(qr,n,l))
print(test)

                                                                      ┌──┐                                                           ┌──┐
0: ───X───@───Ry(-0.5π)───@─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
          │   │           │
1: ───X───┼───┼───────────┼───@───Ry(-0.564π)───@───@───Ry(-0.436π)────@────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
          │   │           │   │   │             │   │   │              │
2: ───X───┼───@───────────┼───┼───@─────────────┼───┼───┼──────────────┼@────Ry(-0.667π)───@───@───Ry(-0.5π)───@───@───Ry(-0.333π)────@─────────────────────────────────────────────────────────────────
          │   │           │   │   │             │   │   │              ││    │             │   │   │           │   │   │              │
3: ───────X───@───────

In [50]:
result = simulator.simulate(test)
print(cirq.dirac_notation(result.final_state_vector))

0.22|000111⟩ + 0.22|001011⟩ + 0.22|001101⟩ + 0.22|001110⟩ + 0.22|010011⟩ + 0.22|010101⟩ + 0.22|010110⟩ + 0.22|011001⟩ + 0.22|011010⟩ + 0.22|011100⟩ + 0.22|100011⟩ + 0.22|100101⟩ + 0.22|100110⟩ + 0.22|101001⟩ + 0.22|101010⟩ + 0.22|101100⟩ + 0.22|110001⟩ + 0.22|110010⟩ + 0.22|110100⟩ + 0.22|111000⟩
